# Choosing the Right Tool: A Comprehensive Guide to Reading Microscopy Data in Python

Microscopy data analysis in Python offers a multitude of options, each with its unique strengths and capabilities. Navigating the expansive landscape of tools can be daunting, but selecting the right one can significantly impact your workflow efficiency and data insights. This guide aims to provide a comprehensive overview of the various Python libraries and frameworks available for reading microscopy data. Whether you prioritize speed, versatility, or integration with other data analysis tools, this guide will help you make informed decisions to meet your specific requirements. Explore the possibilities and empower your microscopy data analysis with the right tools for the job.

## Import and Data Path Assignment

To begin our analysis, we first need to import the necessary libraries and assign the path to our data files. This step ensures that we have access to the tools and resources required for the subsequent tasks.

Let's get started by executing the following code:

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
simgsingle = str(imgsingle)
smcts = str(mcts)
sbigtiff = str(bigtiff)

So, we have few options to open microscopy datafiles:

- skimage.io.imread
- skimage.io.imread_collection
- tifffile.TiffFile
- tifffile.TiffSequence
- bioformats.ImageReader

Imagej hyperstack are organized as **TZCYXS**.

Holoview can also be used. Check availability of reading from disk a la memmap.

Bioformats claims the following standard: separate tiff for each channel and for each time point.
Thus a ome.tif would contain a single plane or a zstack. 

What about tiles? Bioformats has some 6D, 7D and 8D working around 5D. But how is exactly defined 5D?

## Skimage and Tifffile

In [ ]:
t1 = skimage.io.imread(img_tile, plugin="tifffile")
t2 = skimage.io.imread(img_void_tile, plugin="tifffile")
t1.shape, t2.shape

In [ ]:
tf1 = tifffile.imread(img_tile)
tf2 = tifffile.imread(img_void_tile)
tf1.shape, tf2.shape

Only for tiff data files provides:

- sequence
- OME metadata
- memmap
- zarr

In [ ]:
fp1glob = "/home/dati/liaisan/data/260816/FLAT/cyan_em10/cyan_em10_p*.tif"
tifs = tifffile.TiffSequence(fp1glob)
d = tifs.asarray()
print(d.shape)
print(tifs.shape)

In [ ]:
with tifffile.TiffFile(img_tile) as tif:
    tag = tif.pages[0].tags["ImageDescription"]

tag.value

## nima_io.read

| function   |  time | Pos in md | extra md | no errors | note                     |
|------------|-------|-----------|----------|-----------|--------------------------|
| read       |  4.68 | yes       | no       | no        |                          |
| read2      |  12.4 | no        | yes      | no        |                          |
| read_inf   |  4.46 | no        | no       | yes       | core md misses many keys |
| read_bf    | 0.487 | no        | no       | no        |                          |
| reaf_jb    |  4.47 | yes       | no       | no        |                          |
| read_wrap  |     - | yes       | no       | yes       | It calls read            |
|----
| read_pims  |  2.57 | yes       | no       | yes       | extra pims DIMS          |
| read_jpype | 0.263 | yes       | yes      | yes       |                          |

Summary:

- The Jpype approach stands out for its thoroughness and performance.
- Scyjava, built on Jpype, is now superseding it.
- Functions like read_inf and read_bf may be candidates for removal from the convenient library.

In the following, various bioformats implementations are explored. While python-bioformats produces a verbose warning log through python-javabridge, the numerous metadata linked to individual planes are not returned by the external library. Hence, I decided to develop a small library to handle additional (neglected) metadata, such as acquisition stage position (essential for reconstructing tiled images) and illumination and emission settings.

### read

In [ ]:
ir.ensure_vm()

In [ ]:
r = ir.read(simg_void_tile)

In [ ]:
r

### read2

In [ ]:
%memit ir.read2(simg_void_tile)
r2 = ir.read2(simg_void_tile)

In [ ]:
r2

In [ ]:
np.save("extra_md_read2.npy", r2[0])

### read_inf

In [ ]:
%timeit ir.read_inf(img_void_tile)
%memit ir.read_inf(img_void_tile)
rinf = ir.read_inf(img_void_tile)
rinf

### read_bf - bioformats (std)

In [ ]:
%timeit ir.read_bf(simg_void_tile)

In [ ]:
%memit ir.read_bf(simg_void_tile)

In [ ]:
rbf = ir.read_bf(simg_void_tile)

In [ ]:
rbf

### javabridge

In [ ]:
%timeit ir.read_jb(simg_void_tile)

In [ ]:
%memit ir.read_jb(simg_void_tile)

In [ ]:
rjb = ir.read_jb(simg_void_tile)

In [ ]:
rjb

### Stitch

In [ ]:
f = ir.stitch(r[0], r[1], c=2, t=2)
skimage.io.imshow(f)

In [ ]:
r[0]["series"][0]["PositionXYZ"]